In [9]:
import torch
from torch import nn

import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [10]:
# Defining the device = cuda to run the code on gpu
if torch.cuda.is_available():
    device = torch.device("cuda")

In [11]:
# Download mnist fashion train and test datasets
mnist_train = datasets.FashionMNIST(root="./data", download=True, train=True, transform=ToTensor())
mnist_test = datasets.FashionMNIST(root="./data", download=False, train=True, transform=ToTensor())

In [12]:
from torch.utils.data import DataLoader

# Converting to dataloaders
train_dataloder = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloder = DataLoader(mnist_test, batch_size=32, shuffle=True)

In [13]:
# Defining the model
model = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=(3, 3), padding=1, padding_mode="reflect"), # CNN 2 dimensional layer with a 3x3 filter
    nn.MaxPool2d(kernel_size=2), # Max pooling to reduce the dimension and turn less complex the nn
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(588, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
).to(device)

# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Defining the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# Reshaping the data (validation)
for X, y in train_dataloder:
    print(X.shape)
    print(y.shape)
    
    y = nn.functional.one_hot(y, num_classes=10).type(torch.float32)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])
torch.Size([32, 10])


In [15]:
for i in range(10):
    loss_sum = 0
    for X, y in train_dataloder:
        X = X.to(device)
        y = nn.functional.one_hot(y, num_classes=10).type(torch.float32).to(device)

        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(loss_sum)

1043.1445444077253
737.0386180952191
643.1245753355324
582.7083284184337
529.1299956738949
491.7301901243627
467.76725324615836
438.04009353742003
418.20533907972276
398.38306922093034


In [16]:
model.eval()
with torch.no_grad():
    accurate = 0
    total = 0
    for X, y in test_dataloder:
        X = X.to(device)
        y = y.to(device)

        y_pred = nn.functional.softmax(model(X), dim=1)
        correct_pred = (y == y_pred.max(dim=1).indices)

        total += correct_pred.size(0)
        accurate += correct_pred.type(torch.int).sum().item()

    print(f"Accuracy: {(accurate/total)*100:.2f} %")

Accuracy: 92.41 %
